In [1]:
# generate_gemini_sentiment.py

# Import libraries
import pandas as pd
from typing import Dict
import sys
import os
import json # Import json at the top

# Setting secret credentials
from dotenv import load_dotenv  # pip install python-dotenv
load_dotenv()
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


# Local imports
sys.path.append('../src')
from simplify_prompts import simplify_with_gemini
from mt_prompts import translate_with_gemini

# Import Google GenAI libraries
from google import genai
from google.genai import types
from google.api_core import retry # pip install google-api-core

# --- Global Setup ---
# This section is for configurations that are set once.

# Configure retry logic for API calls to handle temporary errors
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [2]:
api_key = os.getenv("GOOGLE_API_KEY")  # make sure your .env has: GOOGLE_API_KEY=your_key_here

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file")

# --- Setup Gemini client ---
client = genai.Client(api_key=api_key)

### Text Simplification for Target Groups

In [3]:


# --- Sample text ---
sample_text = "In a democracy, citizens are responsible for participating in public discussions and voting to shape the policies of their government."

# --- Try each style for each audience ---
audiences = ["low-literacy", "children", "ADHD"]
styles = ["one-shot", "few-shot", "cot"]

results = {}

for aud in audiences:
    results[aud] = {}
    for st in styles:
        print(f"\n🔹 Audience: {aud} | Style: {st}")
        simplified = simplify_with_gemini(client, sample_text, st, aud)
        results[aud][st] = simplified
        print(simplified)



🔹 Audience: low-literacy | Style: one-shot
Democracy needs people to talk to each other. People vote to pick leaders. This helps make rules for everyone.


🔹 Audience: low-literacy | Style: few-shot
People live in a democracy. They must talk to each other. They vote to change the rules.


🔹 Audience: low-literacy | Style: cot
Here's a simplified version of the text:

1.  Democracy is important.
2.  People vote in a democracy.
3.  Voting helps make rules.
4.  People talk about rules too.
5.  This helps the government.


🔹 Audience: children | Style: one-shot
Democracy is when people talk about what's important. They also vote! Voting helps choose the rules for everyone.


🔹 Audience: children | Style: few-shot
Simplified: Democracy is like a game where everyone plays! We talk about things together. Then we vote to make the rules!


🔹 Audience: children | Style: cot
Here's a simple explanation of democracy for kids:

1.  Democracy is like a team.
2.  Everyone on the team gets a say.
3. 

### Multilingual Machine Translation for Target Groups

| Source → Target                              | Audience                                                                   | Language Level / Notes | Prompt Guidance / Considerations                                                                                |
| -------------------------------------------- | -------------------------------------------------------------------------- | ---------------------- | --------------------------------------------------------------------------------------------------------------- |
| Spanish → Ukrainian                          | Immigrants from Ukraine                                                    | B1 Spanish             | Use simple sentences, avoid idioms, clarify meaning. Keep neutral tone.                                         |
| Spanish → German                             | Immigrant Germans                                                          | B2 Spanish             | Use standard German, clear sentence structure, preserve meaning.                                                |                            |                         |
| Spanish → English                            | Non-native English-speaking immigrants in Spain                            | B1-B2 Spanish          | Clear sentences, simple vocabulary. Suitable for official communications, school instructions, healthcare info. |
| Spanish → Polish                             | Polish workers or families in Spain                                        | B1 Spanish             | Focus on workplace instructions or social integration materials. Clear, short sentences.                        |
| Spanish → Simplified Spanish / Plain Spanish | Low literacy adults, older immigrants, or reading-challenged populations   | B1 or lower            | Simplification rather than translation. Short, clear sentences. Plain language.                                 |


the MT prompt should consider:

Proficiency in the source language (B1/B2) → avoid overly complex sentences or idioms.

Cultural and linguistic adaptation → some words may need localization.

Clarity for non-native speakers → shorter sentences, neutral vocabulary.

In [3]:
# --- Sample text ---
sample_text = "Los ciudadanos deben participar en las discusiones públicas y votar para elegir políticas justas."

# --- Define target audiences and language pairs ---
targets = [
    {"source":"Spanish", "target":"Ukrainian", "audience":"Immigrants from Ukraine", "level":"B1", "notes":"Use simple sentences, avoid idioms, keep neutral tone."},
    {"source":"Spanish", "target":"German", "audience":"Immigrant Germans", "level":"B2", "notes":"Use standard German, clear sentence structure."},
    {"source":"Spanish", "target":"English", "audience":"Non-native English-speaking immigrants in Spain", "level":"B1-B2", "notes":"Suitable for official communications, school instructions, healthcare info."},
    {"source":"Spanish", "target":"Polish", "audience":"Polish workers or families in Spain", "level":"B1", "notes":"Focus on workplace instructions or social integration materials."},
    {"source":"Spanish", "target":"Simplified Spanish", "audience":"Low literacy adults or older immigrants", "level":"B1 or lower", "notes":"Short, clear sentences. Plain language."},
]

styles = ["zero-shot", "one-shot", "cot"]

# --- Run demo ---
results = {}

for t in targets:
    aud = t["audience"]
    results[aud] = {}
    for style in styles:
        print(f"\n🔹 Audience: {aud} | Style: {style} | Target: {t['target']}")
        translated = translate_with_gemini(
            client=client,
            text=sample_text,
            style=style,
            source_lang=t["source"],
            target_lang=t["target"],
            audience=aud,
            level=t["level"],
            notes=t["notes"]
        )
        results[aud][style] = translated
        print(translated)



🔹 Audience: Immigrants from Ukraine | Style: zero-shot | Target: Ukrainian
Here's a translation appropriate for Ukrainian immigrants with a B1 level of Spanish:

**Translation:**

Громадяни повинні брати участь в обговореннях важливих питань. І громадяни повинні голосувати, щоб вибрати справедливі правила та закони.


🔹 Audience: Immigrants from Ukraine | Style: one-shot | Target: Ukrainian
Translation:

Громадяни повинні брати участь в обговореннях для всіх людей та голосувати, щоб обрати правильні правила.


🔹 Audience: Immigrants from Ukraine | Style: cot | Target: Ukrainian
Here's a breakdown of the translation process and the final translation itself:

**Step-by-step process:**

1. **Key points in Spanish:**
    * Citizens (Los ciudadanos)
    * Must participate (deben participar)
    * Public discussions (en las discusiones públicas)
    * And vote (y votar)
    * To choose fair policies (para elegir políticas justas)

2. **Simplified sentences (if needed, in Spanish - in this c